In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
supply = pd.read_csv('20201007_da_co.processed.csv').set_index('id')
nsupply = len(supply)
print(supply.shape)
supply.head()

(3617, 3)


,node,capacity (MW),offer ($/MW)
id,,,
A3153_0,North,1.0,8.16
A3153_1,North,245.0,9.54
A3153_2,North,1.0,14.27
A3153_3,North,72.0,14.28
A3165_0,Central,180.0,10.77


In [3]:
nodes = list(supply['node'].unique())
nnodes = len(nodes)
print(nodes)

['North', 'Central', 'South']


In [4]:
demand = pd.read_csv('20201007_bids_cb.processed.csv').set_index('id')
assert demand.node.isin(nodes).all()
ndemand = len(demand)
print(demand.shape)
demand.head()

(309, 3)


,node,demand (MW),bid ($/MW)
id,,,
122073561,North,6.0,2000.0
767746013,North,14.0,2000.0
122073033,Central,1228.0,2000.0
122073042,North,1644.0,2000.0
450523929,Central,132.4,2000.0


In [5]:
demand.groupby(demand['bid ($/MW)'] >= 2000.)['node'].count()

bid ($/MW)
False     41
True     268
Name: node, dtype: int64

In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 5e3, 1e5] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
North-Central,North,Central,5000.0,100000.0
North-South,North,South,5000.0,100000.0
Central-South,Central,South,5000.0,100000.0


In [7]:
rev_lines = lines.copy()
rev_lines.source = lines.dest
rev_lines.dest = lines.source
bi_lines = pd.concat([lines, rev_lines]).set_index(['source', 'dest'])
n_bi_lines = len(bi_lines)
bi_lines


capacity (MW)  susceptance (S)
source  dest                                   
North   Central         5000.0         100000.0
        South           5000.0         100000.0
Central South           5000.0         100000.0
        North           5000.0         100000.0
South   North           5000.0         100000.0
        Central         5000.0         100000.0

In [8]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns')

,demand (MW),capacity (MW)
node,,
Central,28181.2,85287.0
North,12854.2,52353.8
South,16425.5,47877.5


In [9]:
M = Model('dual')

log = StringIO()
M.setLogHandler(log)

muD = M.variable('muD', ndemand, Domain.greaterThan(0.))
muG = M.variable('muG', nsupply, Domain.greaterThan(0.))
etaup = M.variable('etaup', n_bi_lines, Domain.greaterThan(0.))
etadown = M.variable('etadown', n_bi_lines, Domain.greaterThan(0.))
lambdaN = M.variable('lambdaN', nnodes, Domain.unbounded())
gamma = M.variable('gamma', 1, Domain.unbounded())

demand_idx = [nodes.index(x) for x in demand.node]
dldpd = Expr.sub(muD, demand['bid ($/MW)'].values)
dldpd = Expr.add(dldpd, lambdaN.pick(demand_idx))
pd_const = M.constraint('dldpd', dldpd, Domain.greaterThan(0.))

supply_idx = [nodes.index(x) for x in supply.node]
dldpg = Expr.add(muG, supply['offer ($/MW)'].values)
dldpg = Expr.sub(dldpg, lambdaN.pick(supply_idx))
pg_const = M.constraint('dldpg', dldpg, Domain.greaterThan(0.))

theta_eqs = []
for n, n_node in enumerate(nodes):
    if n == 0:
        dldtheta = gamma
    else:
        dldtheta = Expr.zeros(1)
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)

        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        m_n_idx = bi_lines.index.get_loc((m_node, n_node))
        nm_duals = Expr.sub(etaup.index(n_m_idx), 
                            etaup.index(m_n_idx))
        nm_duals = Expr.sub(nm_duals, etadown.index(n_m_idx))
        nm_duals = Expr.add(nm_duals, etadown.index(m_n_idx))
        nm_duals = Expr.add(nm_duals, lambdaN.index(n))
        nm_duals = Expr.sub(nm_duals, lambdaN.index(m))
        dldtheta = Expr.add(
            dldtheta, 
            Expr.mul(bi_lines.loc[(n_node, m_node), 'susceptance (S)'],
                     nm_duals))
    theta_eqs.append(
        M.constraint('dldtheta%d' % n, dldtheta, Domain.equalsTo(0.)))

obj = Expr.add(Expr.dot(muD, demand['demand (MW)'].values), 
               Expr.dot(muG, supply['capacity (MW)'].values))
for n, n_node in enumerate(nodes):
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)
        capacity = bi_lines.loc[(n_node, m_node), 'capacity (MW)']
        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        obj = Expr.add(obj, Expr.mul(capacity, 
                                     Expr.add(etaup.index(n_m_idx),
                                              etadown.index(n_m_idx))))
M.objective('obj', ObjectiveSense.Minimize, obj)
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal or True:
    M.writeTask('proj1dual.opf')

demand['consumed (MW)'] = pd_const.dual()
supply['supplied (MW)'] = pg_const.dual()
buses = pd.DataFrame({
    'node price ($/MW)': lambdaN.level()
}, index=nodes)

In [10]:
summary = pd.concat([
    buses['node price ($/MW)'],
    demand.groupby('node')['consumed (MW)'].sum(),
    supply.groupby('node')['supplied (MW)'].sum(),
], axis=1)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
North,-0.00,12854.2,22854.2
Central,11.84,28178.1,23178.1
South,11.32,16367.5,11367.5


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
North   &         -0.00 &        12854.2 &        22854.2 \\
Central &         11.84 &        28178.1 &        23178.1 \\
South   &         11.32 &        16367.5 &        11367.5 \\
\bottomrule
\end{tabular}



In [11]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

Energy scheduled: 57399.8
Objective value: 1.13911e+08


,node price ($/MW)
North,-0.00
Central,11.84
South,11.32


,node,demand (MW),bid ($/MW),consumed (MW)
id,,,,
1656071284,South,58.0,4.21,0.0
1656072147_0,South,27.0,11.33,27.0
122073523_7,Central,3.1,11.47,0.0
122073523_6,Central,3.1,11.99,3.1
122073523_5,Central,3.1,12.51,3.1
...,...,...,...,...
816860633,Central,5.0,2000.00,5.0
1601140898,Central,3.0,2000.00,3.0
1066937520,Central,30.0,2000.00,30.0


,node,capacity (MW),offer ($/MW),supplied (MW)
id,,,,
A14605_0,North,1.0,-50.0,1.0
A6994_0,North,1.0,-50.0,1.0
A7548_0,North,1.0,-50.0,1.0
A6027_0,North,1.0,-50.0,1.0
A6999_0,North,1.0,-50.0,1.0
...,...,...,...,...
A14896_1,North,69.0,999.0,0.0
A14517_1,Central,19.0,999.0,0.0
A12762_1,North,23.0,999.0,0.0


In [12]:
print(log.getvalue())

Problem
  Name                   : dual            
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 3929            
  Cones                  : 0               
  Scalar variables       : 3943            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   : dual            
  Objective sense        : min             
  Type                   : LO (linear optimization 